# Transfer

In [1]:
%load_ext jupyter_black
%matplotlib inline

In [2]:
import os
from pathlib import Path

import mmmpy


root = Path(os.path.abspath(__name__)).parents[1]
root

PosixPath('/workspaces/mmmpy')

In [3]:
netcdf, binary, grib = tuple(root.glob("data/*.zip"))
netcdf, binary, grib

(PosixPath('/workspaces/mmmpy/data/mosaic3d_tile6_20130531-231500.netcdf.zip'),
 PosixPath('/workspaces/mmmpy/data/mrms_binary_data.zip'),
 PosixPath('/workspaces/mmmpy/data/mrms_grib_data.zip'))

In [4]:
mrms = mmmpy.read_mrms(netcdf)
mrms

<xarray.Dataset>
Dimensions:       (Ht: 31, Lat: 2001, Lon: 2001)
Dimensions without coordinates: Ht, Lat, Lon
Data variables:
    mrefl_mosaic  (Ht, Lat, Lon) int16 dask.array<chunksize=(31, 2001, 2001), meta=np.ndarray>
    Height        (Ht) float32 dask.array<chunksize=(31,), meta=np.ndarray>
Attributes:
    TypeName:        mrefl_mosaic
    DataType:        LatLonHeightGrid
    Time:            1370042100
    FractionalTime:  0.0
    MissingData:     -999.0
    RangeFolded:     -1000.0
    Latitude:        40.0
    Longitude:       -110.0
    Height:          500.0
    LatGridSpacing:  0.01
    LonGridSpacing:  0.01
    attributes:

In [6]:
import xarray as xr
from typing import Iterable


@mmmpy.use.unzip()
def mfread_cfgrib(files: Iterable[Path], args):
    ds = xr.open_mfdataset(
        tuple(files)[:2],
        engine="cfgrib",
        concat_dim=["heightAboveSea"],
        combine="nested",
        chunks={},
        # backend_kwargs=None
        backend_kwargs=dict(
            mask_and_scale=True,
            decode_times=True,
            concat_characters=True,
            decode_coords=True,
            # drop_variables=None,
            use_cftime=None,
            decode_timedelta=None,
            lock=None,
            indexpath="{path}.{short_hash}.idx",
            filter_by_keys={},
            read_keys=[],
            encode_cf=("parameter", "time", "geography", "vertical"),
            squeeze=True,
            time_dims=("time", "step"),
        ),
    )
    return ds


mrms = mfread_cfgrib(grib)
mrms

ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored


<xarray.Dataset>
Dimensions:         (heightAboveSea: 2, latitude: 3500, longitude: 7000)
Coordinates:
    time            datetime64[ns] 2015-02-17T23:58:00
    step            timedelta64[ns] 00:00:00
  * heightAboveSea  (heightAboveSea) float64 1.1e+04 1.25e+03
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      datetime64[ns] 2015-02-17T23:58:00
Data variables:
    unknown         (heightAboveSea, latitude, longitude) float32 dask.array<chunksize=(1, 3500, 7000), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-08-18T22:47 GRIB to CDM+CF via cfgrib-0.9.1...